<a href="https://colab.research.google.com/github/NaokiTano22/Study/blob/main/webscrayping_oreilly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

from bs4 import BeautifulSoup

import re
import csv

In [2]:
html = urlopen("http://www.pythonscraping.com/pages/page1.html")
bsObj = BeautifulSoup(html.read())
print(bsObj.h1)

<h1>An Interesting Title</h1>


URLに関するエラー対策

In [3]:
try:
  html = urlopen("https://pythonscrapingthisurldoesnotexist.com")
except HTTPError as e:
  print(e)
except URLError as e:
  print(e)
else:
  print("It worked!")

<urlopen error [Errno -2] Name or service not known>


存在しないタグを参照したときのエラー対策

In [4]:
try:
  badContent = bsObj.nonExistingTag.anotherTag
except AttributeError as e:
  print("Tag was not found")
else:
  if badContent == None:
    print("Tag was not found")
  else:
    print(badContent)

Tag was not found


<ipython-input-4-7b5c0c9f4671>:2: DeprecationWarning: .nonExistingTag is deprecated, use .find("nonExisting") instead. If you really were looking for a tag called nonExistingTag, use .find("nonExistingTag")
  badContent = bsObj.nonExistingTag.anotherTag


エラー対策も含めたタイトルを取得するコードの例

In [5]:
def getTitle(url):
  try:
    html = urlopen(url)
  except HTTPError as e:
    print(e)
    return None
  try:
    bsObj = BeautifulSoup(html.read())
    title = bsObj.body.h1
  except AttributeError as e:
    return None
  return title


In [6]:
title = getTitle("http://www.pythonscraping.com/pages/page1.html")

findAllの使用例

In [7]:
html = urlopen("http://www.pythonscraping.com/pages/warandpeace.html")
bsObj = BeautifulSoup(html)

nameList = bsObj.findAll("span", {"class":"green"})
nameList[0]

<span class="green">Anna
Pavlovna Scherer</span>

childrenの使用例（一個下のレベルのタグを取得、descendantsは下にあるすべてのタグを取得
）

In [8]:
html = urlopen("http://www.pythonscraping.com/pages/page3.html")
bsObj = BeautifulSoup(html)
childs = []

for child in bsObj.find("table", {"id":"giftList"}).children:
  childs.append(child)
childs[3]

<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>

同じ階層のタグを取得するnext_siblingsの使用例。
似たものとして他にprevious_siblings, next_sibling, previous_siblingなどがある。なお、他に親のタグを呼び出す関数も存在する。（.parent)

In [9]:
sibs = []
for sibling in bsObj.find("table",{"id":"giftList"}).tr.next_siblings:
  sibs.append(sibling)
sibs[1]

<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>

正規表現を利用した必要な情報の抽出
例では"../img/gifts/img*.jpg"を取り出している（*)は任意の文字

In [11]:
images = bsObj.findAll("img", {"src": re.compile("\.\.\/img\/gifts/img.*\.jpg")})
for image in images:
  print(image["src"])

../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


.attrsでは対象とそのタグを辞書形式で返すため、タグを確認することができる。

In [14]:
image.attrs

{'src': '../img/gifts/img6.jpg'}

In [22]:
csvFile = open("test.csv", "w+", newline='')
try:
  writer = csv.writer(csvFile)
  writer.writerow(('number', 'number plus 2', 'number times 2'))
  for i in range(10):
    writer.writerow((i, i+2, i*2))
finally:
  csvFile.close()